In [ ]:
# imports
import pyomo.environ as pe
import pyomo.opt as po

# Inputs

In [10]:
# -------------------------------------------------------------------------
# SETS
# -------------------------------------------------------------------------
factories = ['Ijmuiden', 'Segal', 'South Wales']
customer_areas = ['Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte']
long_bar_types = ['1', '2']
rebar_types = ['A', 'B', 'C']
numOfPeriods = 4
vehicles = ['V_IJ', 'V_SE', 'V_SW']


# Parameters needed for steel capacity/fleet capacity calculations
# length per long bar type
length_lb = {'1': 9, '2': 12}

# length per rebar type
length_rb = {'A': 2.4, 'B': 3.6, 'C': 4.2}

# diameter for rebar and long bar types
diameter = {'1': 0.057, '2': 0.057, 'A': 0.057, 'B': 0.057, 'C': 0.057}

# density of steel in t/m3
density = 7.85

# ------------------------------------------------------------------------
# Steel and Vehicle capactity
# -------------------------------------------------------------------------
# Steel capacity per factory per period in tons
steel_capacity = {'Ijmuiden': 12, 'Segal': 10, 'South Wales': 28}
vehicle_capacity = 10  # in tons


# -------------------------------------------------------------------------
# Transportation costs
# -------------------------------------------------------------------------
# Fixed cost per fleet per period per factory
f_fleet = {'Ijmuiden': 130, 'Segal': 150, 'South Wales': 100}

# variable costs per km 
c_fleet = {'Ijmuiden': 3, 'Segal': 3, 'South Wales': 3}

# -------------------------------------------------------------------------
# Demand 
# -------------------------------------------------------------------------

# demand for each rebar type per period per customer area
demand = {
    # Rebar A
    ('A', 1, 'Bochum'): 2,  ('A', 1, 'Boenen'): 4,  ('A', 1, 'Dortmund'): 2,  ('A', 1, 'Gelsenkirchen'): 5,
    ('A', 1, 'Hagen'): 19,  ('A', 1, 'Iserlohn'): 13, ('A', 1, 'Neuss'): 20, ('A', 1, 'Schwerte'): 4,

    ('A', 2, 'Bochum'): 6,  ('A', 2, 'Boenen'): 8,  ('A', 2, 'Dortmund'): 7,  ('A', 2, 'Gelsenkirchen'): 5,
    ('A', 2, 'Hagen'): 23,  ('A', 2, 'Iserlohn'): 19, ('A', 2, 'Neuss'): 16, ('A', 2, 'Schwerte'): 5,

    ('A', 3, 'Bochum'): 5,  ('A', 3, 'Boenen'): 5,  ('A', 3, 'Dortmund'): 6,  ('A', 3, 'Gelsenkirchen'): 5,
    ('A', 3, 'Hagen'): 25,  ('A', 3, 'Iserlohn'): 17, ('A', 3, 'Neuss'): 14, ('A', 3, 'Schwerte'): 3,

    ('A', 4, 'Bochum'): 3,  ('A', 4, 'Boenen'): 10, ('A', 4, 'Dortmund'): 5,  ('A', 4, 'Gelsenkirchen'): 5,
    ('A', 4, 'Hagen'): 16,  ('A', 4, 'Iserlohn'): 14, ('A', 4, 'Neuss'): 26, ('A', 4, 'Schwerte'): 4,

    # Rebar B
    ('B', 1, 'Bochum'): 4,  ('B', 1, 'Boenen'): 5,  ('B', 1, 'Dortmund'): 4,  ('B', 1, 'Gelsenkirchen'): 9,
    ('B', 1, 'Hagen'): 15,  ('B', 1, 'Iserlohn'): 22, ('B', 1, 'Neuss'): 12, ('B', 1, 'Schwerte'): 2,

    ('B', 2, 'Bochum'): 5,  ('B', 2, 'Boenen'): 8,  ('B', 2, 'Dortmund'): 5,  ('B', 2, 'Gelsenkirchen'): 10,
    ('B', 2, 'Hagen'): 33,  ('B', 2, 'Iserlohn'): 26, ('B', 2, 'Neuss'): 23, ('B', 2, 'Schwerte'): 8,

    ('B', 3, 'Bochum'): 7,  ('B', 3, 'Boenen'): 12, ('B', 3, 'Dortmund'): 8,  ('B', 3, 'Gelsenkirchen'): 6,
    ('B', 3, 'Hagen'): 31,  ('B', 3, 'Iserlohn'): 20, ('B', 3, 'Neuss'): 30, ('B', 3, 'Schwerte'): 2,

    ('B', 4, 'Bochum'): 8,  ('B', 4, 'Boenen'): 13, ('B', 4, 'Dortmund'): 10, ('B', 4, 'Gelsenkirchen'): 6,
    ('B', 4, 'Hagen'): 33,  ('B', 4, 'Iserlohn'): 27, ('B', 4, 'Neuss'): 30, ('B', 4, 'Schwerte'): 6,

    # Rebar C
    ('C', 1, 'Bochum'): 6,  ('C', 1, 'Boenen'): 6,  ('C', 1, 'Dortmund'): 7,  ('C', 1, 'Gelsenkirchen'): 10,
    ('C', 1, 'Hagen'): 12,  ('C', 1, 'Iserlohn'): 14, ('C', 1, 'Neuss'): 22, ('C', 1, 'Schwerte'): 5,

    ('C', 2, 'Bochum'): 7,  ('C', 2, 'Boenen'): 10, ('C', 2, 'Dortmund'): 6,  ('C', 2, 'Gelsenkirchen'): 9,
    ('C', 2, 'Hagen'): 35,  ('C', 2, 'Iserlohn'): 25, ('C', 2, 'Neuss'): 32, ('C', 2, 'Schwerte'): 6,

    ('C', 3, 'Bochum'): 7,  ('C', 3, 'Boenen'): 15, ('C', 3, 'Dortmund'): 4,  ('C', 3, 'Gelsenkirchen'): 9,
    ('C', 3, 'Hagen'): 33,  ('C', 3, 'Iserlohn'): 23, ('C', 3, 'Neuss'): 31, ('C', 3, 'Schwerte'): 7,

    ('C', 4, 'Bochum'): 7,  ('C', 4, 'Boenen'): 12, ('C', 4, 'Dortmund'): 12, ('C', 4, 'Gelsenkirchen'): 10,
    ('C', 4, 'Hagen'): 38,  ('C', 4, 'Iserlohn'): 24, ('C', 4, 'Neuss'): 31, ('C', 4, 'Schwerte'): 2,
}

# -------------------------------------------------------------------------
# Distance between factories and customer areas in km
# -------------------------------------------------------------------------
dist = {

    # From IJmuiden
    ('IJmuiden', 'IJmuiden'): 0,
    ('IJmuiden', 'Segal'): 284,
    ('IJmuiden', 'South Wales'): 826,
    ('IJmuiden', 'Bochum'): 250,
    ('IJmuiden', 'Boenen'): 282,
    ('IJmuiden', 'Dortmund'): 266,
    ('IJmuiden', 'Gelsenkirchen'): 234,
    ('IJmuiden', 'Hagen'): 289,
    ('IJmuiden', 'Iserlohn'): 299,
    ('IJmuiden', 'Neuss'): 259,
    ('IJmuiden', 'Schwerte'): 279,

    # From Segal
    ('Segal', 'IJmuiden'): 284,
    ('Segal', 'Segal'): 0,
    ('Segal', 'South Wales'): 750,
    ('Segal', 'Bochum'): 203,
    ('Segal', 'Boenen'): 242,
    ('Segal', 'Dortmund'): 222,
    ('Segal', 'Gelsenkirchen'): 198,
    ('Segal', 'Hagen'): 206,
    ('Segal', 'Iserlohn'): 226,
    ('Segal', 'Neuss'): 140,
    ('Segal', 'Schwerte'): 216,

    # From South Wales
    ('South Wales', 'IJmuiden'): 826,
    ('South Wales', 'Segal'): 750,
    ('South Wales', 'South Wales'): 0,
    ('South Wales', 'Bochum'): 866,
    ('South Wales', 'Boenen'): 914,
    ('South Wales', 'Dortmund'): 885,
    ('South Wales', 'Gelsenkirchen'): 859,
    ('South Wales', 'Hagen'): 903,
    ('South Wales', 'Iserlohn'): 913,
    ('South Wales', 'Neuss'): 843,
    ('South Wales', 'Schwerte'): 901,

    # From Bochum
    ('Bochum', 'IJmuiden'): 250,
    ('Bochum', 'Segal'): 203,
    ('Bochum', 'South Wales'): 866,
    ('Bochum', 'Bochum'): 0,
    ('Bochum', 'Boenen'): 55,
    ('Bochum', 'Dortmund'): 21,
    ('Bochum', 'Gelsenkirchen'): 19,
    ('Bochum', 'Hagen'): 41,
    ('Bochum', 'Iserlohn'): 51,
    ('Bochum', 'Neuss'): 56,
    ('Bochum', 'Schwerte'): 39,

    # From Boenen
    ('Boenen', 'IJmuiden'): 282,
    ('Boenen', 'Segal'): 242,
    ('Boenen', 'South Wales'): 914,
    ('Boenen', 'Bochum'): 55,
    ('Boenen', 'Boenen'): 0,
    ('Boenen', 'Dortmund'): 34,
    ('Boenen', 'Gelsenkirchen'): 56,
    ('Boenen', 'Hagen'): 44,
    ('Boenen', 'Iserlohn'): 54,
    ('Boenen', 'Neuss'): 102,
    ('Boenen', 'Schwerte'): 32,

    # From Dortmund
    ('Dortmund', 'IJmuiden'): 266,
    ('Dortmund', 'Segal'): 222,
    ('Dortmund', 'South Wales'): 885,
    ('Dortmund', 'Bochum'): 21,
    ('Dortmund', 'Boenen'): 34,
    ('Dortmund', 'Dortmund'): 0,
    ('Dortmund', 'Gelsenkirchen'): 34,
    ('Dortmund', 'Hagen'): 21,
    ('Dortmund', 'Iserlohn'): 36,
    ('Dortmund', 'Neuss'): 75,
    ('Dortmund', 'Schwerte'): 15,

    # From Gelsenkirchen
    ('Gelsenkirchen', 'IJmuiden'): 234,
    ('Gelsenkirchen', 'Segal'): 198,
    ('Gelsenkirchen', 'South Wales'): 859,
    ('Gelsenkirchen', 'Bochum'): 19,
    ('Gelsenkirchen', 'Boenen'): 56,
    ('Gelsenkirchen', 'Dortmund'): 34,
    ('Gelsenkirchen', 'Gelsenkirchen'): 0,
    ('Gelsenkirchen', 'Hagen'): 56,
    ('Gelsenkirchen', 'Iserlohn'): 66,
    ('Gelsenkirchen', 'Neuss'): 62,
    ('Gelsenkirchen', 'Schwerte'): 54,

    # From Hagen
    ('Hagen', 'IJmuiden'): 289,
    ('Hagen', 'Segal'): 206,
    ('Hagen', 'South Wales'): 903,
    ('Hagen', 'Bochum'): 41,
    ('Hagen', 'Boenen'): 44,
    ('Hagen', 'Dortmund'): 21,
    ('Hagen', 'Gelsenkirchen'): 56,
    ('Hagen', 'Hagen'): 0,
    ('Hagen', 'Iserlohn'): 20,
    ('Hagen', 'Neuss'): 66,
    ('Hagen', 'Schwerte'): 14,

    # From Iserlohn
    ('Iserlohn', 'IJmuiden'): 299,
    ('Iserlohn', 'Segal'): 226,
    ('Iserlohn', 'South Wales'): 913,
    ('Iserlohn', 'Bochum'): 51,
    ('Iserlohn', 'Boenen'): 54,
    ('Iserlohn', 'Dortmund'): 36,
    ('Iserlohn', 'Gelsenkirchen'): 66,
    ('Iserlohn', 'Hagen'): 20,
    ('Iserlohn', 'Iserlohn'): 0,
    ('Iserlohn', 'Neuss'): 85,
    ('Iserlohn', 'Schwerte'): 15,

    # From Neuss
    ('Neuss', 'IJmuiden'): 259,
    ('Neuss', 'Segal'): 140,
    ('Neuss', 'South Wales'): 843,
    ('Neuss', 'Bochum'): 56,
    ('Neuss', 'Boenen'): 102,
    ('Neuss', 'Dortmund'): 75,
    ('Neuss', 'Gelsenkirchen'): 62,
    ('Neuss', 'Hagen'): 66,
    ('Neuss', 'Iserlohn'): 85,
    ('Neuss', 'Neuss'): 0,
    ('Neuss', 'Schwerte'): 75,

    # From Schwerte
    ('Schwerte', 'IJmuiden'): 279,
    ('Schwerte', 'Segal'): 216,
    ('Schwerte', 'South Wales'): 901,
    ('Schwerte', 'Bochum'): 39,
    ('Schwerte', 'Boenen'): 32,
    ('Schwerte', 'Dortmund'): 15,
    ('Schwerte', 'Gelsenkirchen'): 54,
    ('Schwerte', 'Hagen'): 14,
    ('Schwerte', 'Iserlohn'): 15,
    ('Schwerte', 'Neuss'): 75,
    ('Schwerte', 'Schwerte'): 0,
}


# Sets and Parameters

In [ ]:
# MODEL
model = pe.ConcreteModel()

# SETS
# Factories
model.F = pe.Set(initialize=factories)

# Customer areas
model.C = pe.Set(initialize=customer_areas)

# Long bar types
model.L = pe.Set(initialize=long_bar_types)

# Rebar types
model.R = pe.Set(initialize=rebar_types)

# Periods
model.P = pe.RangeSet(1,numOfPeriods)

# Vehicles
model.V = pe.Set(initialize=vehicles)

# All locations (factories and customer areas)
model.N = pe.Set(initialize=factories + customer_areas)  # all locations

# PARAMETERS
# demand for each rebar type per period per customer area
model.demand = pe.Param(model.R, model.P, model.C, initialize=demand)

# distance between factories and customer areas in km
model.dist = pe.Param(model.N, model.N, initialize=dist)

# length per long bar type
model.lb = pe.Param(model.L, initialize=length_lb)

# length per rebar type
model.rb = pe.Param(model.R, initialize=length_rb)

# capacity of steel per factory per period in tons
model.steel_cap = pe.Param(model.F, initialize=steel_capacity)

# capacity of each vehicle in tons
model.vehicle_cap = pe.Param(initialize=vehicle_capacity)

# fixed cost per fleet per period per factory
model.fixed_ship = pe.Param(model.F, initialize=f_fleet)

#fixed cost per fleet per period per factory
model.cost_km = pe.Param(model.F, initialize=c_fleet)